In [63]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pandas as pd
import numpy as np
import os

%matplotlib inline

path = '/'.join(os.getcwd().split('/')[:-1])
print path

/Volumes/Dropbox-1/largetransfer/luc/carter


In [64]:
tci = pd.read_csv(path+'/data/model_data/tci_2_2.csv')
ppns = set(tci['ppn'])
ppns_num = set(tci['ppn'].apply(lambda x: x.replace('-','')))

## Foreclosure data

In [65]:
fc = pd.read_csv(path+'/data/clean_data/foreclosure_filings.csv', parse_dates=[2])

In [66]:
fc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45055 entries, 0 to 45054
Data columns (total 26 columns):
parcel            45055 non-null object
status            45055 non-null object
filedate          45055 non-null datetime64[ns]
defendant         45055 non-null object
caseno            45055 non-null object
parcel_address    44986 non-null object
dateid            45055 non-null object
FORE              45055 non-null int64
LATITUDE          45035 non-null float64
LONGITUDE         45035 non-null float64
zip_fore          43920 non-null float64
case_number       45055 non-null object
ctitle            45055 non-null object
cdesig            45055 non-null object
judge             45055 non-null object
magistrate        41211 non-null object
room              407 non-null object
n_action          408 non-null object
f_location        45053 non-null object
lstatus           45055 non-null object
lstatusdt         45055 non-null object
ldispos           45055 non-null object
ldisp

In [67]:
fc.head(2)

,parcel,status,filedate,defendant,caseno,parcel_address,dateid,FORE,LATITUDE,LONGITUDE,...,room,n_action,f_location,lstatus,lstatusdt,ldispos,ldisposdt,pamount,detaildt,plaintiff
0,107-16-046,Inactive,2006-10-03,"AHMAD HAKIM, ET AL",BR-06-000005,BLAINE AVENUE,03/21/2015,1,41.51486,-81.625548,...,NaN,NaN,DF-ROOM 45,INACTIVE,03/07/2007,DEFAULT,02/28/2007,$.00,12/24/2014,"JAMES ROKAKIS, AS TREASURER"
1,119-01-007,Inactive,2006-10-03,IDOLA SMITH AKA IDOLA BINION ET.AL.,BR-06-000018,1827 EAST 79TH STREET,03/21/2015,1,41.50848,-81.633084,...,NaN,NaN,DF-ROOM 45,INACTIVE,12/11/2007,DEFAULT,12/10/2007,$.00,03/21/2015,"JAMES ROKAKIS, AS TREASURER"


In [68]:
fc['pamount'] = fc['pamount'].apply(lambda x: float(x[1:].replace(',','')))

In [69]:
tem = fc.loc[fc.status=='Active',['parcel']]
tem['active_fc'] = 1
tem = tem.drop_duplicates()

In [70]:
tci = pd.merge(tci, tem, \
             how='left', left_on='ppn', right_on='parcel')

In [71]:
tem = fc.loc[fc.filedate>dt.datetime(2011, 3, 1),['parcel','pamount']].groupby('parcel').first()
tem.columns = ['foreclosure_flag']

tci = pd.merge(tci, tem, \
             how='left', left_on='ppn', right_index=True)

In [72]:
tci.loc[(tci.ppn.isin(set(fc[(fc.ldispos=='DIS. W/O PREJ')].parcel))),'fc_dis_w/o_prej'] = 1

In [73]:
tci.loc[(tci.ppn.isin(set(fc[(fc.ldispos=='NEWLY FILED')].parcel))),'fc_newly_filed'] = 1

In [74]:
for dis in ['DEFAULT']:
    for i in xrange(2006,2016):
        tci.loc[(tci.ppn.isin(set(fc[(fc.ldispos==dis) & (fc.filedate.apply(lambda x: x.year) ==i)].parcel))),'fc_default'] = i
tci.loc[tci.fc_default.isnull(),'fc_default'] = 0

In [75]:
dis = 'DISP.OTHER'
for year in xrange(2006,2015):
    tci.loc[tci.ppn.isin(set(fc[(fc.ldispos==dis) & (fc.filedate.apply(lambda x: x.year) ==year)].parcel)),'fc_dispother'] = year
tci.loc[tci.fc_dispother.isnull(),'fc_dispother']= 0

In [76]:
for dis in set(fc.ldispos):
    a = tci[(tci.ppn.isin(set(fc[(fc.ldispos==dis) & (fc.filedate > dt.datetime(2014,3,1))].parcel))) & (tci.vacant>-1)]
    print(dis, len(a), sum(a.vacant))

('DIS. W/PREJ', 1, 0)
('DISP. TRANSCT', 0, 0)
('DISP.OTHER', 217, 129)
('NEWLY FILED', 255, 197)
('DEFAULT', 443, 154)
('UNAVAILABLE', 0, 0)
('DISPOSED', 0, 0)
('BANKRPT/C.O.A. STAY', 45, 6)
('DIS. W/O PREJ', 372, 53)
('SATISFIED', 0, 0)
('DISP.JURY TRIAL', 0, 0)
('MAGISTRATE', 1226, 284)
('ADR', 167, 7)
('DISP.COURT TRIAL', 0, 0)
('STAY', 0, 0)
('RET. STAY', 0, 0)


## Sheriff's auction

In [77]:
sa = pd.read_csv(path+'/data/clean_data/sheriff_auction.csv', parse_dates=[2], encoding="ISO-8859-1")

In [78]:
sa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45843 entries, 0 to 45842
Data columns (total 25 columns):
parcel        45843 non-null object
address       45700 non-null object
salesdt       45843 non-null datetime64[ns]
saleno        45842 non-null float64
sold_amt      28609 non-null object
appraisal     37038 non-null object
minbid        37050 non-null object
scaseno       45843 non-null object
withdrawn     17813 non-null object
plaintiff     45831 non-null object
sdefend       45840 non-null object
purchaser     31260 non-null object
paddress      42891 non-null object
rparty        35478 non-null object
raddress      35478 non-null object
attorney      44680 non-null object
descript      40222 non-null object
location      23362 non-null object
defend_att    5857 non-null object
with          17813 non-null object
status        4508 non-null object
ssold         31381 non-null object
SHF           45843 non-null int64
LATITUDE      45831 non-null float64
LONGITUDE     45831 

In [79]:
sa.head(2)

,parcel,address,salesdt,saleno,sold_amt,appraisal,minbid,scaseno,withdrawn,plaintiff,...,attorney,descript,location,defend_att,with,status,ssold,SHF,LATITUDE,LONGITUDE
0,001-02-016,11436 EDGEWATER DRIVE,2009-08-24,46,NaN,"$225,000","$150,000",CV08664602,Withdrawn - Order of,INDYMAC BANK FSB C/O INDYMAC BANK FSB,...,REIMER/DENNIS/,A SINGLE BRICK DWELLING WITH ATTACHED TWO CAR ...,NaN,NaN,WITHD,NaN,NaN,1,41.489578,-81.764549
1,001-02-016,11436 EDGEWATER DRIVE,2009-11-23,78,NaN,"$225,000","$150,000",CV08664602,Withdrawn - Order of,INDYMAC BANK FSB C/O INDYMAC BANK FSB,...,REIMER/DENNIS/,A SINGLE BRICK DWELLING WITH ATTACHED TWO CAR ...,NaN,NaN,WITHD,NaN,NaN,1,41.489578,-81.764549


In [80]:
def parse_amount(x):
    return float(str(x)[1:].replace(',',''))

In [81]:
sa.loc[sa.appraisal.notnull(), 'appraisal'] = sa.loc[sa.appraisal.notnull(), 'appraisal'].apply(parse_amount)

In [82]:
sa['ssold'] = sa.ssold.apply(lambda x: 1 if x=='Yes' else 0)

In [83]:
tem = sa.loc[sa.salesdt>dt.datetime(2014, 1, 1),['parcel','ssold']].groupby('parcel').sum()
tem.columns = ['auction']
tem.loc[tem['auction'].isnull(),'auction'] = 0
tci = pd.merge(tci, tem, \
              how='left', left_on='ppn', right_index=True)
tci.loc[tci['auction'].isnull(),'auction'] = 0
tci.loc[(tci.vacant>-1),['auction','vacant']].groupby('auction').agg([sum,len])

vacant       
           sum    len
auction              
0         8556  98711
1          577   1695
2            2      3

In [84]:
tci[tci.auction.isnull()] = 0
tci[tci.auction > 0] = 1

## Tax bill

In [85]:
tb = pd.read_csv(path+'/data/clean_data/taxbill_dec14.csv', dtype=object)

In [86]:
tb.head()

,TAX_BILL_ID,TAX_YEAR,WHS_ID,PROPERTY_ID,PPN,PROPERTY_NUMBER,MACHINE_ID,EFFECTIVE_STATUS,VALUE_CHANGE_TIMESTAMP,PROPERTY_TYPE,...,LOCATION_UNIT_NUMBER,HOMESTEAD_ASSESSED_VALUE,SURPLUS_PAYMENT,OMITTED_TAX_PENALTY_1ST,OMITTED_TAX_PENALTY_2ND,AUDITOR_REFUND,TRASURER_REFUND,TIF_AMOUNT_1ST,TIF_AMOUNT_2ND,UPDATE_DATE
0,284393728,2014,4287556,603364178,00707138,007-07-138,NaN,NaN,NaN,800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12/09/2014
1,284393729,2014,4287558,103364182,00707140,007-07-140,NaN,NaN,NaN,800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12/09/2014
2,284393730,2014,4287559,403364185,00707141,007-07-141,NaN,NaN,NaN,800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12/09/2014
3,284393731,2014,4287560,703364188,00707142,007-07-142,NaN,NaN,NaN,800,...,NaN,NaN,5.23,NaN,NaN,NaN,NaN,0,0,12/09/2014
4,284393733,2014,4287562,403364194,00707144,007-07-144,NaN,NaN,NaN,800,...,NaN,8750,NaN,NaN,NaN,NaN,NaN,0,0,12/09/2014


In [87]:
tb = tb.fillna(0)

In [88]:
# plt.hist(t.loc[(t.vacant==0),'GRAND_TOTAL_BALANCE'].apply(float).values, bins = np.linspace(0,20000),alpha=0.5,normed=True);
# plt.hist(t.loc[(t.vacant==1),'GRAND_TOTAL_BALANCE'].apply(float).values, bins = np.linspace(0,20000),alpha=0.5,normed=True);

In [89]:
# t.loc[(t.vacant>-1), ['LENDER_PROCESS_TYPE','vacant']].groupby('LENDER_PROCESS_TYPE').agg([sum,len])

In [90]:
# t.loc[t.LENDER_PROCESS_TYPE==0,'LENDER_PROCESS_TYPE'] = 'None'

In [91]:
tci = pd.merge(tci, tb[['PROPERTY_NUMBER','LENDER_PROCESS_TYPE','GRAND_TOTAL_BALANCE']], left_on='ppn', right_on='PROPERTY_NUMBER',how='left')

In [92]:
tci.loc[tci.LENDER_PROCESS_TYPE==0,'LENDER_PROCESS_TYPE'] = 'None'

In [93]:
tci.shape

(110154, 109)

In [94]:
tci.to_csv(path+'/data/model_data/tci_2_3.csv',index=False)

In [95]:
tci.shape

(110154, 109)